In [12]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch3d as p3d
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, load_ply, save_ply
from pytorch3d.loss import chamfer_distance

In [13]:
### config

# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 200
batch_size = 15

sxxxxes = os.listdir("../../data/all_results/")
train_sxxxxes = sxxxxes[:372]
test_sxxxxes = sxxxxes[372:]

In [14]:
# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.fc2 = nn.Linear(64, 256)  # 隐藏层1到隐藏层2
        self.fc3 = nn.Linear(256, 512)  # 隐藏层2到隐藏层3
        self.fc4 = nn.Linear(512, 256)  # 隐藏层3到隐藏层4
        self.fc5 = nn.Linear(256, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


# DataSet类
class TrainingSet(Dataset):
    def __init__(self, s_name_list, transform=None, sample_num=5000) -> None:
        self.input = [
            f"../../data/all_results/{sname}/0_sphere.obj" for sname in s_name_list
        ]
        self.target = [
            f"../../data/all_results/{sname}/{sname}.ply" for sname in s_name_list
        ]
        self.sample_num = sample_num
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        #对input采样5000个点
        input_tensor_points, input_tensor_faces, _ = load_obj(self.input[index], load_textures=False)
        input_mesh = Meshes([input_tensor_points], [input_tensor_faces.verts_idx]) #注意这里有个.verts_idx
        input_tensor_points_sampled = sample_points_from_meshes(input_mesh,5000).squeeze(0)
        #对output采样5000个点
        target_tensor_points, target_tensor_faces = load_ply(self.target[index])
        target_mesh = Meshes([target_tensor_points],[target_tensor_faces])
        target_tensor_points_sampled = sample_points_from_meshes(target_mesh).squeeze(0)

        return input_tensor_points_sampled.to(device),target_tensor_points_sampled.to(device)

    def get_target(self, input):
        raise NotImplemented

In [15]:
#################################################################################
# pts,faces = load_ply(f"../../data/all_results/s0004_pulmonary_artery.nii.g_1/s0004_pulmonary_artery.nii.g_1.ply")
# mymesh = Meshes([pts],[faces])
# a = sample_points_from_meshes(mymesh,5)
# a = a.squeeze(0)
# a
# mymesh
#################################################################################

In [16]:
def plot_pointcloud(points, title=""):
    """Sample points uniformly from the surface of the mesh."""
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [17]:
my_train_dataset = TrainingSet(train_sxxxxes)
data_loader = DataLoader(my_train_dataset, batch_size=batch_size, shuffle=False)


In [18]:
# 创建模型实例
model = PointCloudFCNet(input_dim, output_dim).to(device)
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=3, bias=True)
)

In [19]:
# 定义损失函数和优化器
criterion = chamfer_distance
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器

In [20]:
next(iter(data_loader))

[tensor([[[ 0.0750, -0.0151,  0.0634],
          [-0.3546, -0.3758,  0.2541],
          [-0.4337, -0.2710,  0.1684],
          ...,
          [-0.0123,  0.0367,  0.1373],
          [ 0.1583, -0.0191, -0.1322],
          [ 0.3453, -0.2538,  0.0267]],
 
         [[ 0.2570,  0.0861,  0.0868],
          [ 0.3904, -0.3220,  0.0590],
          [-0.1913,  0.4343, -0.0899],
          ...,
          [ 0.2725, -0.2485,  0.0707],
          [-0.1331,  0.2137, -0.0343],
          [-0.3554,  0.6676, -0.3018]],
 
         [[ 0.3714, -0.0536,  0.1230],
          [ 0.6549, -0.1877, -0.2958],
          [ 0.0641,  0.4419,  0.1158],
          ...,
          [ 0.8132, -0.1644, -0.1694],
          [ 0.0108, -0.0797,  0.3038],
          [ 0.0030,  0.4445, -0.1004]],
 
         ...,
 
         [[ 0.0079,  0.6045, -0.2121],
          [ 0.8088, -0.2320, -0.1193],
          [-0.0547,  0.3439,  0.1796],
          ...,
          [-0.0062,  0.5559,  0.1128],
          [ 0.6188, -0.1804, -0.3567],
          [ 0.7017

In [21]:
# 训练模型

for epoch in range(num_epochs):
    for input_point_cloud,target_point_cloud in data_loader:
        optimizer.zero_grad()
        output_point_cloud = model(input_point_cloud)  # 前向传播
        loss,_ = criterion(output_point_cloud, target_point_cloud)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 优化模型参数

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    # if (epoch + 1) % 5 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # plot_pointcloud(output_point_cloud)# 绘制点云

Epoch [1/200], Loss: 0.0366
Epoch [2/200], Loss: 0.0164
Epoch [3/200], Loss: 0.0143
Epoch [4/200], Loss: 0.0130
Epoch [5/200], Loss: 0.0120
Epoch [6/200], Loss: 0.0111
Epoch [7/200], Loss: 0.0104
Epoch [8/200], Loss: 0.0092
Epoch [9/200], Loss: 0.0085
Epoch [10/200], Loss: 0.0081
Epoch [11/200], Loss: 0.0078
Epoch [12/200], Loss: 0.0075
Epoch [13/200], Loss: 0.0073
Epoch [14/200], Loss: 0.0071
Epoch [15/200], Loss: 0.0070
Epoch [16/200], Loss: 0.0068
Epoch [17/200], Loss: 0.0066
Epoch [18/200], Loss: 0.0065
Epoch [19/200], Loss: 0.0064
Epoch [20/200], Loss: 0.0063
Epoch [21/200], Loss: 0.0062
Epoch [22/200], Loss: 0.0061
Epoch [23/200], Loss: 0.0061
Epoch [24/200], Loss: 0.0060
Epoch [25/200], Loss: 0.0059
Epoch [26/200], Loss: 0.0059
Epoch [27/200], Loss: 0.0058
Epoch [28/200], Loss: 0.0057
Epoch [29/200], Loss: 0.0056
Epoch [30/200], Loss: 0.0056
Epoch [31/200], Loss: 0.0056
Epoch [32/200], Loss: 0.0055
Epoch [33/200], Loss: 0.0055
Epoch [34/200], Loss: 0.0055
Epoch [35/200], Loss: 0

In [22]:
# 使用训练好的模型生成新的点云数据
try_input_point_cloud,_,_ = load_obj("../../data/all_results/s1272_pulmonary_artery.nii.g_1/0_sphere.obj", load_textures=False)
new_point_cloud = model(try_input_point_cloud.to(device))
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

tensor([[-0.0462,  0.0878,  0.2990],
        [-0.0555,  0.1179,  0.2942],
        [-0.0738,  0.1157,  0.3005],
        ...,
        [ 0.8420, -0.1950, -0.2843],
        [ 0.8241, -0.1848, -0.2849],
        [ 0.7968, -0.2203, -0.2924]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [23]:
#看一眼新的输出长啥样
save_ply("./9-30-2113.ply",new_point_cloud)